# Risk Management And Insurance

## Data Pull NoteBook

### By: Aaron Bromeland, Bou Gazley, Mitch Lauer, Sven Wilson

#### Introduction 

The data was pulled from two locations. The first one was LinkedIn and the second source of the data was Handshake. The structures of the sites made it necessary for two different scripts to be created. Additionally, a selenium web driver must be installed for this to work on another computer. The computer this was run on requred a chrome driver but others are available.

Data Pulls - Required
1. LinkedIn - Not Logged In
2. Handshake - Will need a uiowa login to run this one


#### Import the needed Modules


In [18]:
# Import Needed Modules
import requests
import pandas as pd
import numpy as np
import json
import re
from lxml import html
import time
#%pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import glob
import os


## Data Expoloration/Learning about Industry

### LifeCycle for Insurance Claim
1. Core Ops Programs
    - Sales or Agency - Insurance Agents
    - Origination - New Business Function - Application - New Business Processor
    - Operational Support - Underwriters
    - Premiums Collection - Premiums Analyst
    - Claims - Claims Examiners
    - Regulatory and Compliance
    - Customer Service
    - Commissions and Licensing
    - **New Product Development - Actuaries** - Not what we would be looking for:
2. Overhead Functions
    - Finance and Accounting/IT
    - Corporate Overhead Functions - Legal/Finance/Accounting/HR
    - New Grad Training Program
    - Marketing Functions
    
    
### Certifications
A. Series 65


## LinkedIn Datasource

Adjust keywords and Locations for URL search in the next steps


In [2]:
# LinkedIn Search Terms
keywords = ['Insurance%20Legal',
            'Insurance%20Marketing',
            'Insurance%20New%20Grad%20Training',
            'Insurance%20HR',
            'Insurance%20Accounting%20and%20Finance',
            'Insurance%20New%20Product%20Development',
            'Insurance%20Licensing',
            'Insurance%20Commissions',
            'Customer%20Service',
            'Insurance%20REgulatory%20and%20Compliance',
            'Claims%20Examiner',
            'Insurance%20Claims%20Analyst',
            'Premiums%20Analyst',
            'Premiums%20Collection',
            'Underwriters',
            'Insurance%20Agent',
            'New%20Business%20Processor',
            'Insurance%20Sales',
            'Insurance%20Origination']

# LinkedIn Locations
locations = {
    'Iowa':'location=Iowa%2C%20United%20States&locationId=&geoId=103078544',
    'Illinois':'location=Illinois%2C%20United%20States&locationId=&geoId=101949407',
    'Missouri':'location=Missouri%2C%20United%20States&locationId=&geoId=101486475',
    'Nebraska':'location=Nebraska%2C%20United%20States&locationId=&geoId=101197782',
    'South Dakota':'location=South%20Dakota%2C%20United%20States&locationId=&geoId=100115110',
    'North Dakota':'location=North%20Dakota%2C%20United%20States&locationId=&geoId=104611396',
    'Minnesota':'location=Minnesota%2C%20United%20States&locationId=&geoId=103411167',
    'Wisconsin':'location=Wisconsin%2C%20United%20States&locationId=&geoId=104454774',
    'Indiana':'location=Indiana%2C%20United%20States&locationId=&geoId=103336534',
    'Michigan':'location=Michigan%2C%20United%20States&locationId=&geoId=103051080',
    'Ohio':'location=Ohio%2C%20United%20States&locationId=&geoId=106981407',
    'Kentucky':'location=Kentucky%2C%20United%20States&locationId=&geoId=106470801',
    'Tennessee':'location=Tennessee%2C%20United%20States&locationId=&geoId=104629187'   
}

### Run All keyword and Location Combinations

This code cell will run all the keyword and locations combinations above. This can take some time to run, and will need a steady internet connection throughout. The printout will give you status and how many jobs were found for the combination.

In [15]:
dfAll = pd.DataFrame(columns = ['Title','Company','ID','link','location','State','Keyword'])
for key,value in locations.items():
    for keyword in keywords:
        state = key
        ksearch = keyword.replace("%20",' ')
        url = f'https://www.linkedin.com/jobs/search?keywords={keyword}&{value}&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0'
        print(state)
        print(ksearch)
        print(url)
        
        # Open URL and get listing of jobs
        wd = webdriver.Chrome('chromedriver.exe')
        wd.get(url)
        #wd.maximize_window()
        time.sleep(30)
        previous_height = wd.execute_script('return document.body.scrollHeight')
        jobCount = [s.text for s in wd.find_elements(By.XPATH,'//*[@id="main-content"]/div/h1/span[1]')]
        if len(jobCount)>0:
            print(f'The job Count for this page is {jobCount[0]}')
        while True:
            wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            time.sleep(5)
            wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            try:
                wd.execute_script('document.getElementsByClassName("infinite-scroller__show-more-button infinite-scroller__show-more-button--visible")[0].click();')
            except:
                pass
            time.sleep(5)
            new_height = wd.execute_script('return document.body.scrollHeight')
            if new_height == previous_height:
                break
            previous_height = new_height

        # Get list of jobs with links
        jobTitle = [s.text for s in wd.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li/div/a')]
        jobCompany = [s.text for s in wd.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li/div/div[2]/h4/a')]
        jobCompanyLink = [s.get_attribute('href') for s in wd.find_elements(By.XPATH,'//*[@id="main-content"]/section[2]/ul/li/div/div[2]/h4/a')]
        jobID = [re.search('\d+$',s.get_attribute('data-entity-urn')).group(0) for s in wd.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li/div')]
        jobLocation = [' '.join([x.text for x in s.find_elements(By.CLASS_NAME, 'job-search-card__location')]) for s in wd.find_elements(By.XPATH,'//*[@id="main-content"]/section[2]/ul/li/div')]
        jobLink = [s.get_attribute('href') for s in wd.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li/div/a')]
        print(len(jobTitle))
        print(len(jobCompany))
        print(len(jobCompanyLink))
        print(len(jobLocation))
        print(f'The number of jobs pulled from the webpage: {len(jobLink)}')
        df = pd.DataFrame({'Title':jobTitle,'Company':jobCompany,'ID':jobID,'link':jobLink,'location':jobLocation})
        df['State'] = state
        df['Keyword'] = ksearch
        df.to_csv(f"Jobs_{state}_{ksearch.replace(' ','_')}.csv")
        dfAll.append(df)
        print(f'End of Processing for {state} - {ksearch}')
        

Iowa
Insurance Legal
https://www.linkedin.com/jobs/search?keywords=Insurance%20Legal&location=Iowa%2C%20United%20States&locationId=&geoId=103078544&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0


<ipython-input-15-15194a5e1df2>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('chromedriver.exe')


The job Count for this page is 201
201
201
201
201
The number of jobs pulled from the webpage: 201
End of Processing for Iowa - Insurance Legal
Iowa
Insurance Marketing
https://www.linkedin.com/jobs/search?keywords=Insurance%20Marketing&location=Iowa%2C%20United%20States&locationId=&geoId=103078544&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 273
230
230
230
230
The number of jobs pulled from the webpage: 230
End of Processing for Iowa - Insurance Marketing
Iowa
Insurance New Grad Training
https://www.linkedin.com/jobs/search?keywords=Insurance%20New%20Grad%20Training&location=Iowa%2C%20United%20States&locationId=&geoId=103078544&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 2
2
2
2
2
The number of jobs pulled from the webpage: 2
End of Processing for Iowa - Insurance New Grad Training
Iowa
Insurance HR
https://www.linkedin.com/jobs/search?keywords=Insurance%20HR&location=Iowa%2C%20United%20States&locationId=&geoId=10

The job Count for this page is 77
77
77
77
77
The number of jobs pulled from the webpage: 77
End of Processing for Illinois - Insurance New Product Development
Illinois
Insurance Licensing
https://www.linkedin.com/jobs/search?keywords=Insurance%20Licensing&location=Illinois%2C%20United%20States&locationId=&geoId=101949407&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 241
189
189
189
189
The number of jobs pulled from the webpage: 189
End of Processing for Illinois - Insurance Licensing
Illinois
Insurance Commissions
https://www.linkedin.com/jobs/search?keywords=Insurance%20Commissions&location=Illinois%2C%20United%20States&locationId=&geoId=101949407&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 102
102
102
102
102
The number of jobs pulled from the webpage: 102
End of Processing for Illinois - Insurance Commissions
Illinois
Customer Service
https://www.linkedin.com/jobs/search?keywords=Customer%20Service&location=Illi

The job Count for this page is 23
22
22
22
22
The number of jobs pulled from the webpage: 22
End of Processing for Missouri - Insurance REgulatory and Compliance
Missouri
Claims Examiner
https://www.linkedin.com/jobs/search?keywords=Claims%20Examiner&location=Missouri%2C%20United%20States&locationId=&geoId=101486475&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 73
73
73
73
73
The number of jobs pulled from the webpage: 73
End of Processing for Missouri - Claims Examiner
Missouri
Insurance Claims Analyst
https://www.linkedin.com/jobs/search?keywords=Insurance%20Claims%20Analyst&location=Missouri%2C%20United%20States&locationId=&geoId=101486475&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 49
48
48
48
48
The number of jobs pulled from the webpage: 48
End of Processing for Missouri - Insurance Claims Analyst
Missouri
Premiums Analyst
https://www.linkedin.com/jobs/search?keywords=Premiums%20Analyst&location=Missouri%2C%20U

...

The job Count for this page is 118
118
118
118
118
The number of jobs pulled from the webpage: 118
End of Processing for Wisconsin - Insurance Licensing
Wisconsin
Insurance Commissions
https://www.linkedin.com/jobs/search?keywords=Insurance%20Commissions&location=Wisconsin%2C%20United%20States&locationId=&geoId=104454774&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 49
49
49
49
49
The number of jobs pulled from the webpage: 49
End of Processing for Wisconsin - Insurance Commissions
Wisconsin
Customer Service
https://www.linkedin.com/jobs/search?keywords=Customer%20Service&location=Wisconsin%2C%20United%20States&locationId=&geoId=104454774&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 3,000+
997
997
997
997
The number of jobs pulled from the webpage: 997
End of Processing for Wisconsin - Customer Service
Wisconsin
Insurance REgulatory and Compliance
https://www.linkedin.com/jobs/search?keywords=Insurance%20REgulatory%20

The job Count for this page is 58
58
58
58
58
The number of jobs pulled from the webpage: 58
End of Processing for Indiana - Claims Examiner
Indiana
Insurance Claims Analyst
https://www.linkedin.com/jobs/search?keywords=Insurance%20Claims%20Analyst&location=Indiana%2C%20United%20States&locationId=&geoId=103336534&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 145
143
143
143
143
The number of jobs pulled from the webpage: 143
End of Processing for Indiana - Insurance Claims Analyst
Indiana
Premiums Analyst
https://www.linkedin.com/jobs/search?keywords=Premiums%20Analyst&location=Indiana%2C%20United%20States&locationId=&geoId=103336534&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
0
0
0
0
The number of jobs pulled from the webpage: 0
End of Processing for Indiana - Premiums Analyst
Indiana
Premiums Collection
https://www.linkedin.com/jobs/search?keywords=Premiums%20Collection&location=Indiana%2C%20United%20States&locationId=&geoId=103336534&f_TPR=r604

The job Count for this page is 19
19
19
19
19
The number of jobs pulled from the webpage: 19
End of Processing for Michigan - Underwriters
Michigan
Insurance Agent
https://www.linkedin.com/jobs/search?keywords=Insurance%20Agent&location=Michigan%2C%20United%20States&locationId=&geoId=103051080&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 216
25
25
25
25
The number of jobs pulled from the webpage: 25
End of Processing for Michigan - Insurance Agent
Michigan
New Business Processor
https://www.linkedin.com/jobs/search?keywords=New%20Business%20Processor&location=Michigan%2C%20United%20States&locationId=&geoId=103051080&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 13
13
13
13
13
The number of jobs pulled from the webpage: 13
End of Processing for Michigan - New Business Processor
Michigan
Insurance Sales
https://www.linkedin.com/jobs/search?keywords=Insurance%20Sales&location=Michigan%2C%20United%20States&locationId=&geo

4
4
4
4
The number of jobs pulled from the webpage: 4
End of Processing for Ohio - Insurance Origination
Kentucky
Insurance Legal
https://www.linkedin.com/jobs/search?keywords=Insurance%20Legal&location=Kentucky%2C%20United%20States&locationId=&geoId=106470801&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 162
162
162
162
162
The number of jobs pulled from the webpage: 162
End of Processing for Kentucky - Insurance Legal
Kentucky
Insurance Marketing
https://www.linkedin.com/jobs/search?keywords=Insurance%20Marketing&location=Kentucky%2C%20United%20States&locationId=&geoId=106470801&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 315
315
315
315
315
The number of jobs pulled from the webpage: 315
End of Processing for Kentucky - Insurance Marketing
Kentucky
Insurance New Grad Training
https://www.linkedin.com/jobs/search?keywords=Insurance%20New%20Grad%20Training&location=Kentucky%2C%20United%20States&locationId=&geoId=106

The job Count for this page is 134
124
124
124
124
The number of jobs pulled from the webpage: 124
End of Processing for Tennessee - Insurance HR
Tennessee
Insurance Accounting and Finance
https://www.linkedin.com/jobs/search?keywords=Insurance%20Accounting%20and%20Finance&location=Tennessee%2C%20United%20States&locationId=&geoId=104629187&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 186
186
186
186
186
The number of jobs pulled from the webpage: 186
End of Processing for Tennessee - Insurance Accounting and Finance
Tennessee
Insurance New Product Development
https://www.linkedin.com/jobs/search?keywords=Insurance%20New%20Product%20Development&location=Tennessee%2C%20United%20States&locationId=&geoId=104629187&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
The job Count for this page is 19
19
19
19
19
The number of jobs pulled from the webpage: 19
End of Processing for Tennessee - Insurance New Product Development
Tennessee
Insurance Licensing
https

### Consolidate created files - Done in case it needs to run overnight. 

Will need to change the path. If you make df=dfall then you would not need to run the below code block, but this was added because usually the top cell would run overnight and this would consolidate the file for what was needed to continue processing the next day.

In [21]:
all_files = glob.glob(os.path.join(r'C:\Users\ab81004\Jobs', "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df

,Unnamed: 0,Title,Company,ID,link,location,State,Keyword
0,0,Claims Examiner,Homeowner Services Group,3267909391,https://www.linkedin.com/jobs/view/claims-exam...,"Alton, IL",Illinois,Claims Examiner
1,1,Claims Analyst,"The Custom Companies, Inc.",3281175315,https://www.linkedin.com/jobs/view/claims-anal...,"Northlake, IL",Illinois,Claims Examiner
2,2,Spanish Fraud Customer Service (Remote),Discover Financial Services,3278718024,https://www.linkedin.com/jobs/view/spanish-fra...,"Illinois, United States",Illinois,Claims Examiner
3,3,Claims Analyst,Reynolds Consumer Products,3282044939,https://www.linkedin.com/jobs/view/claims-anal...,"Lake Forest, IL",Illinois,Claims Examiner
4,4,Property Claims Adjuster -- Commercial Lines,Liberty Mutual Insurance,3241033739,https://www.linkedin.com/jobs/view/property-cl...,"Chicago, IL",Illinois,Claims Examiner
...,...,...,...,...,...,...,...,...
32164,8,U/W Tech - Natl Accts,Sentry,3274114775,https://www.linkedin.com/jobs/view/u-w-tech-na...,"Stevens Point, WI",Wisconsin,Underwriters
32165,9,System Support Technician,Old Republic Surety Company,3275628326,https://www.linkedin.com/jobs/view/system-supp...,"Brookfield, WI",Wisconsin,Underwriters
32166,10,Customer Care Representative - Personal Lines ...,West Bend Mutual Insurance,3102078559,https://www.linkedin.com/jobs/view/customer-ca...,"West Bend, WI",Wisconsin,Underwriters
32167,11,Accounting Specialist (Hybrid),Old Republic Surety Company,3177057689,https://www.linkedin.com/jobs/view/accounting-...,"Brookfield, WI",Wisconsin,Underwriters


### Pull Details from Each job

This will pull the details for each job url that was collected above. This can be long running as well. Print out will give an update after 10 jobs

In [33]:
allJobs = []
x = 0
for index, row in df.iterrows():
    URL = row['link']
    page = requests.get(URL)
    root = html.fromstring(page.content)
    posts = dict()
    posts['jobTitle'] = row['Title']
    posts['jobCompnay'] = row['Company']
    posts['jobID'] = row['ID']
    posts['jobLink'] = row['link']
    posts['jobLocation'] = row['location']
    posts['jobState'] = row['State']
    posts['jobKeyword'] = row['Keyword']
    posts['jobDetails'] = ''.join([re.sub(r'\t|\n','',s.text_content()).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/div/section/div')])
    posts['jobText'] = [re.sub(r'\t|\n','',s).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/div/section/div/text()')]
    posts['jobBullets'] = [s for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/div/section/div/ul/li/text()')]
    posts['seniorityLevel'] = ''.join([re.sub(r'\t|\n','',s.text_content()).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[1]/span')])
    posts['employmentType'] = ''.join([re.sub(r'\t|\n','',s.text_content()).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[2]/span')])
    posts['jobFunction'] = ''.join([re.sub(r'\t|\n','',s.text_content()).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[3]/span')])
    posts['industry'] = ''.join([re.sub(r'\t|\n','',s.text_content()).strip() for s in root.xpath('//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[4]/span')])
    if x%10 == 0:
        print(x)
    x += 1
    allJobs.append(posts)
    time.sleep(2)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

15510
15520
15530
15540
15550
15560
15570
15580
15590
15600
15610
15620
15630
15640
15650
15660
15670
15680
15690
15700
15710
15720
15730
15740
15750
15760
15770
15780
15790
15800
15810
15820
15830
15840
15850
15860
15870
15880
15890
15900
15910
15920
15930
15940
15950
15960
15970
15980
15990
16000
16010
16020
16030
16040
16050
16060
16070
16080
16090
16100
16110
16120
16130
16140
16150
16160
16170
16180
16190
16200
16210
16220
16230
16240
16250
16260
16270
16280
16290
16300
16310
16320
16330
16340
16350
16360
16370
16380
16390
16400
16410
16420
16430
16440
16450
16460
16470
16480
16490
16500
16510
16520
16530
16540
16550
16560
16570
16580
16590
16600
16610
16620
16630
16640
16650
16660
16670
16680
16690
16700
16710
16720
16730
16740
16750
16760
16770
16780
16790
16800
16810
16820
16830
16840
16850
16860
16870
16880
16890
16900
16910
16920
16930
16940
16950
16960
16970
16980
16990
17000
17010
17020
17030
17040
17050
17060
17070
17080
17090
17100
17110
17120
17130
17140
17150
17160
1717

29170
29180
29190
29200
29210
29220
29230
29240
29250
29260
29270
29280
29290
29300
29310
29320
29330
29340
29350
29360
29370
29380
29390
29400
29410
29420
29430
29440
29450
29460
29470
29480
29490
29500
29510
29520
29530
29540
29550
29560
29570
29580
29590
29600
29610
29620
29630
29640
29650
29660
29670
29680
29690
29700
29710
29720
29730
29740
29750
29760
29770
29780
29790
29800
29810
29820
29830
29840
29850
29860
29870
29880
29890
29900
29910
29920
29930
29940
29950
29960
29970
29980
29990
30000
30010
30020
30030
30040
30050
30060
30070
30080
30090
30100
30110
30120
30130
30140
30150
30160
30170
30180
30190
30200
30210
30220
30230
30240
30250
30260
30270
30280
30290
30300
30310
30320
30330
30340
30350
30360
30370
30380
30390
30400
30410
30420
30430
30440
30450
30460
30470
30480
30490
30500
30510
30520
30530
30540
30550
30560
30570
30580
30590
30600
30610
30620
30630
30640
30650
30660
30670
30680
30690
30700
30710
30720
30730
30740
30750
30760
30770
30780
30790
30800
30810
30820
3083

### Convert dictionary to a pandas data frame

Convert the dictionary created above to a dataframe.

In [34]:
print(url)
print(len(allJobs))
dfJob = pd.DataFrame.from_dict(allJobs)
dfJob

https://www.linkedin.com/jobs/search?keywords=Insurance%20Origination&location=Tennessee%2C%20United%20States&locationId=&geoId=104629187&f_TPR=r604800&f_JT=F&f_E=2&&position=1&pageNum=0
32169


,jobTitle,jobCompnay,jobID,jobLink,jobLocation,jobState,jobKeyword,jobDetails,jobText,jobBullets,seniorityLevel,employmentType,jobFunction,industry
0,Claims Examiner,Homeowner Services Group,3267909391,https://www.linkedin.com/jobs/view/claims-exam...,"Alton, IL",Illinois,Claims Examiner,"In joining Homeowner Services (HOS), you becom...","[, ]",[Understand database applications and operatin...,Entry level,Full-time,Finance and Sales,Consumer Services
1,Claims Analyst,"The Custom Companies, Inc.",3281175315,https://www.linkedin.com/jobs/view/claims-anal...,"Northlake, IL",Illinois,Claims Examiner,The Custom Companies is a Full-Service Transpo...,"[, is a Full-Service Transportation company an...","[Reviews manages, and dispositions cargo claim...",Entry level,Full-time,Strategy/Planning and Information Technology,Truck Transportation
2,Spanish Fraud Customer Service (Remote),Discover Financial Services,3278718024,https://www.linkedin.com/jobs/view/spanish-fra...,"Illinois, United States",Illinois,Claims Examiner,Discover. A brighter future.Great News! You ar...,"[, To receive the bonus, you must be employed ...","[Sign-on Bonus: up to $2K*, Andâ¦ We offer 10...",Entry level,Full-time,Finance and Sales,"Banking, Financial Services, and Hospitality"
3,Claims Analyst,Reynolds Consumer Products,3282044939,https://www.linkedin.com/jobs/view/claims-anal...,"Lake Forest, IL",Illinois,Claims Examiner,Job DescriptionJob Title: Claims AnalystCity: ...,"[, Claims Analyst, Lake Forest /, Illinois, Jo...",[Review policies to determine which charges ar...,Entry level,Full-time,Finance and Sales,Manufacturing
4,Property Claims Adjuster -- Commercial Lines,Liberty Mutual Insurance,3241033739,https://www.linkedin.com/jobs/view/property-cl...,"Chicago, IL",Illinois,Claims Examiner,Property Field Claims AdjusterAdvance your Cla...,"[, Are you looking for an opportunity to join ...","[ Conducts a prompt, thorough and fair investi...",Entry level,Full-time,Finance and Sales,Insurance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32164,U/W Tech - Natl Accts,Sentry,3274114775,https://www.linkedin.com/jobs/view/u-w-tech-na...,"Stevens Point, WI",Wisconsin,Underwriters,Performs underwriting technician duties provid...,[Performs underwriting technician duties provi...,"[Work closely with Underwriters, Underwriter E...",Entry level,Full-time,Accounting/Auditing and Finance,Insurance Agencies and Brokerages
32165,System Support Technician,Old Republic Surety Company,3275628326,https://www.linkedin.com/jobs/view/system-supp...,"Brookfield, WI",Wisconsin,Underwriters,DescriptionOld Republic Surety Company (ORSC) ...,"[, (ORSC) is currently seeking a, to join its,...",[Answer service desk phone calls and monitor s...,Entry level,Full-time,Other,Insurance
32166,Customer Care Representative - Personal Lines ...,West Bend Mutual Insurance,3102078559,https://www.linkedin.com/jobs/view/customer-ca...,"West Bend, WI",Wisconsin,Underwriters,(Please note- we are open to filling this role...,"[, As a licensed agent, handle calls in our Pe...",[Our Customer Care Center is open Monday - Fri...,Entry level,Full-time,Other,Insurance
32167,Accounting Specialist (Hybrid),Old Republic Surety Company,3177057689,https://www.linkedin.com/jobs/view/accounting-...,"Brookfield, WI",Wisconsin,Underwriters,DescriptionOld Republic Surety Company (ORSC) ...,"[, (ORSC) is currently seeking an, to join its...",[Responsible for timely and accurate month-end...,Entry level,Full-time,Accounting/Auditing and Finance,Insurance


### Save Dataframe

Save the dataframe as both a pickled and csv file.

In [37]:
dfJob.to_pickle('LinkedIn_All.pkl')
dfJob.to_csv('LinkedIn_All.csv')
df.to_pickle("Jobs_All.pkl")
df.to_csv("Jobs_All.csv")

### Test Print a job

Test print the job to review the data that was collected

In [38]:
allJobs[3]

{'jobTitle': 'Claims Analyst',
 'jobCompnay': 'Reynolds Consumer Products',
 'jobID': 3282044939,
 'jobLink': 'https://www.linkedin.com/jobs/view/claims-analyst-at-reynolds-consumer-products-3282044939?refId=CrhlLoOxnxOF8poHO2nlnw%3D%3D&trackingId=gDIsvW4omCQsJTGZY3K4tg%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'jobLocation': 'Lake Forest, IL',
 'jobState': 'Illinois',
 'jobKeyword': 'Claims Examiner',
 'jobDetails': 'Job DescriptionJob Title: Claims AnalystCity: Lake Forest / State: IllinoisJob DescriptionJoin Reynolds Consumer Productsâ\x80¦a world of opportunities! We are passionate about achieving results and have fun winning as a team! We are committed to a diverse and inclusive workplace environment in which individual differences are recognized, respected and appreciated.At Reynolds, we provide amazing job opportunities for growth with competitive salaries and benefits in an exciting, dynamic, fast-paced, and fun workplace environment. Are you lookin

## Handshake 
 
Obtain the Handshake data. This requires that a user login so there is a location in the cell below to add a new username and password for the handshake site. The search criteria that I used was for 11K jobs, but the pagination limited the pull to only 10K. The filters might need to be refined in the URL search, and more keyworks would likely lead to better results from this search. 

This cell will run for a long time. It ran for 36 hours during the project. The printout below will give a status for each page the program is on. The good part is it pulls all of the job detail in one loop. It does not work like the LinkedIn data that gathers all of the urls first and then goes back to each url for the job details

In [57]:
# This is a very long running cell and it will error out at 10K records. The last time this was run for 11K records it ran
# for 36 hours.
allJobs = []
for page in range(0,229):
    url = f"https://uiowa.joinhandshake.com/stu/postings?page={page+1}&per_page=50&sort_direction=desc&sort_column=default&employment_type_names%5B%5D=Full-Time&job.job_types%5B%5D=9&majors%5B%5D=77271"
    wd = webdriver.Chrome('chromedriver.exe')
    wd.get(url)
    time.sleep(30)
    wd.execute_script('document.getElementsByClassName("sso-button")[0].click();')
    time.sleep(30)
    # Change user Name and Password
    wd.execute_script('document.getElementById("j_username").value="USERNAME";')
    wd.execute_script('document.getElementById("j_password").value="PASSWORD";')
    wd.execute_script('document.getElementsByClassName("btn btn-primary login-button")[0].click();')
    time.sleep(30)
    print(f"Starting Page: {page+1}")
    for i in range(0,50):
            wd.execute_script(f'document.getElementsByClassName("style__card___3r7ZR")[{i}].click();')
            time.sleep(5)
            try:
                wd.execute_script('document.getElementsByClassName("style__link-button___2YCu0 default-focus style__see-more-link___2who5 style__inherit-color___25tVa")[0].click();')
            except:
                pass
            time.sleep(1)
            try:
                wd.execute_script('document.getElementsByClassName("style__link-button___2YCu0 default-focus style__see-more-link___1jkYF style__inherit-color___25tVa")[0].click();')
            except:
                pass
            time.sleep(1)
            posts = dict()
            posts['jobTitle'] = [s.text for s in wd.find_elements(By.XPATH, '//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/h1')]
            posts['jobCompany'] = [s.text for s in wd.find_elements(By.XPATH, '//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/a')]
            posts['jobLink'] = [s.get_attribute('href') for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/h1/a')]
            posts['jobLocation'] = [s.text for s in wd.find_elements(By.XPATH, '//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/div/div')]
            posts['about'] = [re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div')]
            posts['jobDetails'] = ''.join([re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[2]/div[3]/span')])
            posts['jobText'] = [re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[2]/div[3]/span')]
            posts['jobBullets'] = [s.text for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[2]/div[3]/span/div/div/div/div/div/ul/li')]
            posts['seniorityLevel'] = 'Entry Level'
            posts['employmentType'] = ''.join([re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/text()[0]')])
            posts['company'] = [re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[5]/div[3]/div/div')]
            posts['industry'] = ''.join([re.sub(r'\t|\n','',s.text).strip() for s in wd.find_elements(By.XPATH,'//*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[5]/div[3]/div/div[2]/div/div[2]')])
            time.sleep(2)
            print(i)
            allJobs.append(posts)
            
    
    #wd.execute_script("document.querySelector('[data-hook=\"search-pagination-next\"]').click();")
    time.sleep(30)

<ipython-input-57-97e44aba4081>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('chromedriver.exe')


Starting Page: 1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


46
47
48
49
Starting Page: 53
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 54
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 55
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 56
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 58
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 59
0
1
2
3
4
5
6
7
8
9
10

37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 105
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 106
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 107
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 108
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 109
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 110
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Startin

12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 157
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 158
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 159
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 160
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 161
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Starting Page: 162
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
2

JavascriptException: Message: javascript error: Cannot read properties of undefined (reading 'click')
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x006178B3+2193587]
	Ordinal0 [0x005B0681+1771137]
	Ordinal0 [0x004C41A8+803240]
	Ordinal0 [0x004C6BB4+814004]
	Ordinal0 [0x004C6A72+813682]
	Ordinal0 [0x004C73EA+816106]
	Ordinal0 [0x0051E519+1172761]
	Ordinal0 [0x0050D7AC+1103788]
	Ordinal0 [0x0051DAE2+1170146]
	Ordinal0 [0x0050D5C6+1103302]
	Ordinal0 [0x004E77E0+948192]
	Ordinal0 [0x004E86E6+952038]
	GetHandleVerifier [0x008C0CB2+2738370]
	GetHandleVerifier [0x008B21B8+2678216]
	GetHandleVerifier [0x006A17AA+512954]
	GetHandleVerifier [0x006A0856+509030]
	Ordinal0 [0x005B743B+1799227]
	Ordinal0 [0x005BBB68+1817448]
	Ordinal0 [0x005BBC55+1817685]
	Ordinal0 [0x005C5230+1856048]
	BaseThreadInitThunk [0x76F3FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77867A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77867A6E+238]


### Test Print the length

Used to check the record results

In [58]:
print(len(allJobs))

10000


### Convert the Dictionary to a dataframe

Convert the dictionary of jobs created from scapping the site into a pandas dataframe that can be saved and used in analsis. 

In [59]:
df = pd.DataFrame(allJobs)
df

,jobTitle,jobCompany,jobLink,jobLocation,about,jobDetails,jobText,jobBullets,seniorityLevel,employmentType,company,industry
0,[Entry Level Analyst Development Programs],[Accenture],[https://app.joinhandshake.com/stu/jobs/674027...,"[Milwaukee, WI and 36 more]","[Application deadline, March 31, 2023 10:59 PM...",Kickstart your consulting career by joining on...,[Kickstart your consulting career by joining o...,[Create and deliver platform and custom-design...,Entry Level,,"[Employees25,000+, IndustryManagement Consulti...",Management Consulting
1,[BioPharma Business Associate/ Analyst],"[PharmaACE, LLC]",[https://app.joinhandshake.com/stu/jobs/355406...,"[Washington, DC and 7 more]","[Application deadline, December 31, 2022 11:30...",Welcome Students!Partner with our esteemed cli...,[Welcome Students!Partner with our esteemed cl...,[],Entry Level,,[],
2,[Entry Level – Associate Data Scientist (Comme...,[IBM],[https://app.joinhandshake.com/stu/jobs/692080...,"[Washington, DC and 7 more]","[Application deadline, December 30, 2022 11:00...",IntroductionA career in IBM Consulting embrace...,[IntroductionA career in IBM Consulting embrac...,[Implementing and validating predictive and pr...,Entry Level,,"[Employees25,000+, IndustryElectronic & Comput...",Electronic & Computer Hardware
3,[Client Services Analyst-Summer 2023 start date],[NAILBITER],[https://app.joinhandshake.com/stu/jobs/692092...,"[Chicago, IL and 2 more]","[Application deadline, November 29, 2022 11:00...",Position Overview: We are currently looking fo...,[Position Overview: We are currently looking f...,[Hands-on mentoring and training with Senior M...,Entry Level,,[],
4,[Oliver Wyman - North America P&C Insurance Ac...,[Oliver Wyman Actuarial Consulting],[https://app.joinhandshake.com/stu/jobs/692050...,"[Chicago, IL]","[Application deadline, October 10, 2022 12:00 ...",Company OverviewOliver Wyman is a global leade...,[Company OverviewOliver Wyman is a global lead...,"[Ratemaking, underwriting and claims model dev...",Entry Level,,[],
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,[U220031 - Accounts Payable Associate - U.B. F...,[University at Buffalo Human Resources],[https://app.joinhandshake.com/stu/jobs/643538...,"[Buffalo, NY]","[Application deadline, November 5, 2022 10:45 ...",Position Summary:Your future begins here – gre...,[Position Summary:Your future begins here – gr...,"[Process disbursements for payment including:,...",Entry Level,,[],
9996,[P220155 - Coordinator of Business Affairs - B...,[University at Buffalo Human Resources],[https://app.joinhandshake.com/stu/jobs/643580...,"[Buffalo, NY]","[Application deadline, November 5, 2022 10:45 ...",Position Summary:The Business Affairs Coordina...,[Position Summary:The Business Affairs Coordin...,[Assist Director of Budget and Administrative ...,Entry Level,,[],
9997,[Transitional Youth Behavioral Health Specialist],[Cornerstones of Care],[https://app.joinhandshake.com/stu/jobs/643741...,"[Wichita, KS]","[Application deadline, December 31, 2022 1:55 ...",Position: Transitional Youth Behavioral Health...,[Position: Transitional Youth Behavioral Healt...,[Nonviolence-helping to build safety skills an...,Entry Level,,[],
9998,[Field Guide],[Outback Therapeutic Expeditions],[https://app.joinhandshake.com/stu/jobs/644604...,"[Lehi, UT]","[Application deadline, September 30, 2022 5:10...",,[],[],Entry Level,,[],


### Save the Data

Save the data to a csv and pickle file to be used later. Also the last line will test read in the pickle file that was created.

In [60]:
df.to_csv('Handshake.csv')

In [61]:
df.to_pickle("Handshake.pkl")

In [62]:
df1 = pd.read_pickle("Handshake.pkl")
df1

,jobTitle,jobCompany,jobLink,jobLocation,about,jobDetails,jobText,jobBullets,seniorityLevel,employmentType,company,industry
0,[Entry Level Analyst Development Programs],[Accenture],[https://app.joinhandshake.com/stu/jobs/674027...,"[Milwaukee, WI and 36 more]","[Application deadline, March 31, 2023 10:59 PM...",Kickstart your consulting career by joining on...,[Kickstart your consulting career by joining o...,[Create and deliver platform and custom-design...,Entry Level,,"[Employees25,000+, IndustryManagement Consulti...",Management Consulting
1,[BioPharma Business Associate/ Analyst],"[PharmaACE, LLC]",[https://app.joinhandshake.com/stu/jobs/355406...,"[Washington, DC and 7 more]","[Application deadline, December 31, 2022 11:30...",Welcome Students!Partner with our esteemed cli...,[Welcome Students!Partner with our esteemed cl...,[],Entry Level,,[],
2,[Entry Level – Associate Data Scientist (Comme...,[IBM],[https://app.joinhandshake.com/stu/jobs/692080...,"[Washington, DC and 7 more]","[Application deadline, December 30, 2022 11:00...",IntroductionA career in IBM Consulting embrace...,[IntroductionA career in IBM Consulting embrac...,[Implementing and validating predictive and pr...,Entry Level,,"[Employees25,000+, IndustryElectronic & Comput...",Electronic & Computer Hardware
3,[Client Services Analyst-Summer 2023 start date],[NAILBITER],[https://app.joinhandshake.com/stu/jobs/692092...,"[Chicago, IL and 2 more]","[Application deadline, November 29, 2022 11:00...",Position Overview: We are currently looking fo...,[Position Overview: We are currently looking f...,[Hands-on mentoring and training with Senior M...,Entry Level,,[],
4,[Oliver Wyman - North America P&C Insurance Ac...,[Oliver Wyman Actuarial Consulting],[https://app.joinhandshake.com/stu/jobs/692050...,"[Chicago, IL]","[Application deadline, October 10, 2022 12:00 ...",Company OverviewOliver Wyman is a global leade...,[Company OverviewOliver Wyman is a global lead...,"[Ratemaking, underwriting and claims model dev...",Entry Level,,[],
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,[U220031 - Accounts Payable Associate - U.B. F...,[University at Buffalo Human Resources],[https://app.joinhandshake.com/stu/jobs/643538...,"[Buffalo, NY]","[Application deadline, November 5, 2022 10:45 ...",Position Summary:Your future begins here – gre...,[Position Summary:Your future begins here – gr...,"[Process disbursements for payment including:,...",Entry Level,,[],
9996,[P220155 - Coordinator of Business Affairs - B...,[University at Buffalo Human Resources],[https://app.joinhandshake.com/stu/jobs/643580...,"[Buffalo, NY]","[Application deadline, November 5, 2022 10:45 ...",Position Summary:The Business Affairs Coordina...,[Position Summary:The Business Affairs Coordin...,[Assist Director of Budget and Administrative ...,Entry Level,,[],
9997,[Transitional Youth Behavioral Health Specialist],[Cornerstones of Care],[https://app.joinhandshake.com/stu/jobs/643741...,"[Wichita, KS]","[Application deadline, December 31, 2022 1:55 ...",Position: Transitional Youth Behavioral Health...,[Position: Transitional Youth Behavioral Healt...,[Nonviolence-helping to build safety skills an...,Entry Level,,[],
9998,[Field Guide],[Outback Therapeutic Expeditions],[https://app.joinhandshake.com/stu/jobs/644604...,"[Lehi, UT]","[Application deadline, September 30, 2022 5:10...",,[],[],Entry Level,,[],
